In [1]:
%pip install opencv-python mediapipe

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import mediapipe as mp
import mediapipe.tasks.python.components.containers
import numpy as np
import pandas as pd
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.stattools import adfuller
from itertools import product
import math

In [4]:
def calculate_length(p1 :tuple[int,int] , p2:tuple[int,int]) -> float:
    """
    두 점 사이의 거리를 계산합니다. (유클라디안 거리)
    :param p1: 첫 번째 점 (x, y)
    :param p2: 두 번째 점 (x, y)
    :return: 두 점 사이의 거리
    """
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def calculate_angle(p1:tuple[int,int], p2:tuple[int,int], p3:tuple[int,int]):
    """
    삼각형의 세 점 사이의 각도를 계산합니다.
    :param p1: 첫 번째 점 (x, y)
    :param p2: 두 번째 점 (x, y)
    :param p3: 세 번째 점 (x, y)
    :return: 삼각형의 세 점 사이의 각도 ∠p1p2p3
    """
    # 삼각형의 세 점 사이의 각도를 계산합니다.
    a = calculate_length(p1, p2)
    b = calculate_length(p2, p3)
    c = calculate_length(p3, p1)
    # 코사인 법칙을 사용하여 각도를 계산합니다.
    # ∠abc = arccos((a² + b² - c²) / 2ab)
    return math.degrees(math.acos((a ** 2 + b ** 2 - c ** 2) / (2 * a * b)))

In [ ]:

def extractSkeleton(video_path, output_path):
    # 1. 미디어파이프를 사용하여 비디오에서 스켈레톤 구조 추출하기
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    # video_path = './videos/Standard.MOV' # 스쿼트
    # video_path = './videos/pushup.mp4' # 푸시업
    cap = cv2.VideoCapture(video_path)  # 비디오 파일 경로
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    pose_data = []
    # cnt = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            height, width, _ = image.shape
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                # 왼쪽 팔 각도 계산
                left_elbow_angle = calculate_angle( (landmarks[11].x * width, landmarks[11].y * height), (landmarks[13].x * width, landmarks[13].y * height), (landmarks[15].x * width, landmarks[15].y * height))
                # 오른쪽 팔 각도 계산
                right_elbow_angle = calculate_angle( (landmarks[12].x * width, landmarks[12].y * height), (landmarks[14].x * width, landmarks[14].y * height), (landmarks[16].x * width, landmarks[16].y * height))
                # 왼쪽 손목 각도 계산
                left_wrist_angle = calculate_angle( (landmarks[13].x * width, landmarks[13].y * height), (landmarks[15].x * width, landmarks[15].y * height), (landmarks[17].x * width, landmarks[17].y * height))
                # 오른쪽 손목 각도 계산
                right_wrist_angle = calculate_angle( (landmarks[14].x * width, landmarks[14].y * height), (landmarks[16].x * width, landmarks[16].y * height), (landmarks[18].x * width, landmarks[18].y * height))
                # 왼쪽 어깨 각도 계산
                left_shoulder_angle = calculate_angle( (landmarks[23].x * width, landmarks[23].y * height), (landmarks[11].x * width, landmarks[11].y * height), (landmarks[13].x * width, landmarks[13].y * height))
                # 오른쪽 어깨 각도 계산
                right_shoulder_angle = calculate_angle( (landmarks[24].x * width, landmarks[24].y * height), (landmarks[12].x * width, landmarks[12].y * height), (landmarks[14].x * width, landmarks[14].y * height))
                # 어깨 중앙값 계산
                shoulder_center = (landmarks[11].x * width + landmarks[12].x * width) / 2, (landmarks[11].y * height + landmarks[12].y * height) / 2
                # 엉덩이 중앙값 계산
                hip_center = (landmarks[23].x * width + landmarks[24].x * width) / 2, (landmarks[23].y * height + landmarks[24].y * height) / 2
                # 무릎 중앙값 계산
                knee_center = (landmarks[25].x * width + landmarks[26].x * width) / 2, (landmarks[25].y * height + landmarks[26].y * height) / 2
                # 엉덩이 각도 계산
                hip_angle = calculate_angle(shoulder_center, hip_center, knee_center)
                # 허리 각도 계산
                waist_angle = calculate_angle( (landmarks[23].x * width, landmarks[23].y * height), (landmarks[11].x * width, landmarks[11].y * height), (landmarks[24].x * width, landmarks[24].y * height))
                # 왼쪽 무릎 각도 계산
                left_knee_angle = calculate_angle( (landmarks[23].x * width, landmarks[23].y * height), (landmarks[25].x * width, landmarks[25].y * height), (landmarks[27].x * width, landmarks[27].y * height))
                # 오른쪽 무릎 각도 계산
                right_knee_angle = calculate_angle( (landmarks[24].x * width, landmarks[24].y * height), (landmarks[26].x * width, landmarks[26].y * height), (landmarks[28].x * width, landmarks[28].y * height))
                # 왼쪽 발목 각도 계산
                left_ankle_angle = calculate_angle( (landmarks[25].x * width, landmarks[25].y * height), (landmarks[27].x * width, landmarks[27].y * height), (landmarks[29].x * width, landmarks[29].y * height))
                # 오른쪽 발목 각도 계산
                right_ankle_angle = calculate_angle( (landmarks[26].x * width, landmarks[26].y * height), (landmarks[28].x * width, landmarks[28].y * height), (landmarks[30].x * width, landmarks[30].y * height))

                # 각도를 리스트에 저장
                pose_data.append([left_elbow_angle, right_elbow_angle, left_wrist_angle, right_wrist_angle, left_shoulder_angle, right_shoulder_angle, hip_angle, waist_angle, left_knee_angle, right_knee_angle, left_ankle_angle, right_ankle_angle])
    cap.release()
    # 2. 시계열 데이터로 변환
    pose_df = pd.DataFrame(pose_data)
    pose_df.columns = ['left_elbow_angle', 'right_elbow_angle', 'left_wrist_angle', 'right_wrist_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'hip_angle', 'waist_angle', 'left_knee_angle', 'right_knee_angle', 'left_ankle_angle', 'right_ankle_angle']
    pose_df.to_csv(output_path, index=False)
    return pose_df

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    joint_angles = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # RGB 변환
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # 예시: 왼쪽 팔꿈치의 각도 (왼쪽 어깨, 왼쪽 팔꿈치, 왼쪽 손목을 이용)
            angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            )
            joint_angles.append(angle)

    cap.release()
    return joint_angles

# 비디오 경로 지정
video_paths = ["path_to_save_subvideos/segment_0.avi", "path_to_save_subvideos/segment_1.avi", ...]  # 여기에 모든 세그먼트 경로를 추가

all_angles = {}
for idx, video_path in enumerate(video_paths):
    angles = process_video(video_path)
    all_angles[f"segment_{idx}"] = angles

# 이후 all_angles를 원하는 형식으로 저장하면 됩니다. 예를 들어, CSV나 JSON 파일로 저장 가능합니다.

# 2. 시계열 데이터로 변환
pose_df = pd.DataFrame(pose_data)
pose_df.columns = ['left_elbow_angle', 'right_elbow_angle', 'left_wrist_angle', 'right_wrist_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'hip_angle', 'waist_angle', 'left_knee_angle', 'right_knee_angle', 'left_ankle_angle', 'right_ankle_angle']